## What’s your favorite movie or TV show? - Tutorial on Webscraping IMDB

In this blog post, we are going to look at an interesting application of webscraping!!! We would like to develop a recommender system to see which movie or TV shows share actors with your favorite movie or show. Here is a link to my project repository: https://github.com/Stellawww/whats-your-fav-movie.

## Running through the Navigation Process

To get familiar with the nagivation process our scraper would take, we start by practice clicking through the navigation steps on the IMDB website.
* First, we could pick the favorite movie on IMDB page. Note that its page link is 
`https://www.imdb.com/title/tt3281548/`.
* Next, click on the *Cast & Crew* section, this would take to a page where its url is like `<original_url>fullcredits/`.
* Then, scroll down on the page and click on one of the actors' head photo, this would then direct to another page which contains information of the actor's past work in the *Filmography* section.

Our scraper is going to replicate this process and record actors names and titles of their movies and TV shows.

## Setting up the Project

Open a terminal in the location of your repository and type the following code. This will create a IMDB_scraper project on you laptop.

In [ ]:
conda activate PIC16B
scrapy startproject IMDB_scraper
cd IMDB_scraper

Then, create a file inside the `spiders` directory called `imdb_spider.py` and we can finally start working on our scraper!

## Constructing the Scraper

import the scrapy librabry which allows us to perfrom the web scraping.
We need to define a spider class for scraping. 

In [ ]:
import scrapy

class ImdbSpider(scrapy.Spider):
    name = 'imdb_spider'
    
    # the url of my favorite movie or TV show
    start_urls = ['https://www.imdb.com/title/tt3281548/']


Inside the *ImdbSpider* class, we are going to define three parsing methods: 
* `parse()`
* `parse_full_credits()`
* `parse_actor_page()`

In [ ]:
def parse(self, response):

        """
        This parser method assumes that you are on your favorite movie page.
        It then navigates you to the Cast & Crew page. Once you are there,
        parse_full_credits() method is called. 
        Note that this method returns no data.
        """

        # specify the Caste&Crew's url and request to go to that page

        caste_crew_url = 'https://www.imdb.com/title/tt3281548/fullcredits/'
        yield scrapy.Request(caste_crew_url, callback = self.parse_full_credits)

Assuming that you are on your favorite movie page, the above `parse()` method works by taking you to the *Cast & Crew* page and call the next parser method `parse_full_credits()`.

In [ ]:
def parse_full_credits(self, response):

        """
        This parser method assumes that you are on the Cast & Crew page. 
        It then parse links data of each actor listed on the page and yield a
        request to go to each actor's page. When the actor's page is reached,
        parse_actor_page() method is called.
        Note that this method returns no data.
        """

        # create a list of relative paths corresponding to each actor
        suffix_links = [a.attrib["href"] for a in response.css("td.primary_photo a")]

        # iterate through the suffix_links and request to go to each actor's page
        # and call the next parser method
        for suffix in suffix_links:
            actor_links = "https://www.imdb.com" + suffix
            yield scrapy.Request(actor_links, callback = self.parse_actor_page)

Now after we get to the *Cast & Crew* page, the above `parse_full_credits()` method uses css selectors to get the links to each actor's page. This works as if we are clicking on an actor's headsoht. Then the `scrapy.Request()` takes us to each actor's page and call the next parser method `parse_actor_page()`. 

In [ ]:
def parse_actor_page(self, response):

        """
        This parser method assumes that you are on an actor's page. 
        It then parse the actor's name and movies or TV shows that the actor participated in.
        It yields a dictionary that contains actors' names and corresponding movies.
        """

        # get the actor's name once we are on the actor's page
        actor_name = response.css("span.itemprop::text").get()

        # iterate through the boxes that contains movie or TV titles
        # and get the list of their titles 
        for movie_or_TV_boxes in response.css("div.filmo-row"):

            movie_or_TV_name = [movie_or_TV_boxes.css("a:first-child::text").get()]

            # return the dictionary pairing actor names and movies they worked 
            yield{
            "actor" : actor_name, 
            "movie_or_TV_name" : movie_or_TV_name
            }

Cool now we get to the actor's page, the above `parse_actor_page()` obtains the actor's name and movie/TV names by using css selector on the current response. Finally, it yeids a dictionary containing all of the actor names and movies/TVs that they worked.

## Making Recommendations 

After we are done with constructing the scraper, running `scrapy crawl imdb_spider -o results.csv` in the terminal will save a `.csv` file called "result" in the **IMDB_scraper** folder. To access the resulting data, we would use the `pandas` library to read in the csv file.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("results.csv")

In [3]:
# subsetting the dataframe to keep only the movie / TV names
df = df[["movie_or_TV_name"]]

# rename the column
df = df.rename(columns={"movie_or_TV_name":"Movie or TV Show"})

In [9]:
# return a Series containing counts of unique movie / TV show in df 
# in a descending order
recommend = df.value_counts()

type(recommend)

# reset the index of the recommend Series so that the index is treated as a column
# the "name" object is used to give a name for the column containing the original Series values
recommend = recommend.reset_index(name = "Number of Shared Actors")

pandas.core.series.Series

In [6]:
recommend = recommend.head(10)
recommend

,Movie or TV Show,Number of Shared Actors
0,Little Women,131
1,Old Hearts Cafe,21
2,Castle Rock,20
3,The Oscars,16
4,Kromz Homes: Dreams,15
5,Entertainment Tonight,15
6,Celebrity Page,15
7,Hollywood Insider,14
8,Don't Look Up,14
9,Spenser Confidential,11


The above dataframe shows the top 10 movies / TV shows that share most actors with my favorite movie!!